In [0]:
CREATE OR REPLACE VIEW workspace.project_gold.vw_flight_delays AS
SELECT
    f.flight_key,
    d.full_date,
    d.year,
    d.month,
    d.day,
    a.airline,
    ao.airport AS origin_airport,
    ad.airport AS destination_airport,
    f.departure_delay,
    f.arrival_delay,
    f.cancelled,
    f.diverted
FROM workspace.project_gold.FactTable f
LEFT JOIN workspace.project_silver.DimDate d ON f.date_key = d.date_key
LEFT JOIN workspace.project_silver.DimAirlines a ON f.airline_key = a.airline_key
LEFT JOIN workspace.project_silver.DimAirports ao ON f.origin_airport_key = ao.airport_key
LEFT JOIN workspace.project_silver.DimAirports ad ON f.destination_airport_key = ad.airport_key;

In [0]:
CREATE OR REPLACE VIEW workspace.project_gold.vw_air_traffic AS
SELECT
    d.full_date,
    a.airline,
    COUNT(*) AS flights,
    SUM(CASE WHEN cancelled = 1 THEN 1 ELSE 0 END) AS cancelled_flights,
    SUM(CASE WHEN diverted = 1 THEN 1 ELSE 0 END) AS diverted_flights,
    SUM(distance) AS total_distance
FROM workspace.project_gold.FactTable f
LEFT JOIN workspace.project_silver.DimDate d ON f.date_key = d.date_key
LEFT JOIN workspace.project_silver.DimAirlines a ON f.airline_key = a.airline_key
GROUP BY d.full_date, a.airline;


In [0]:
CREATE OR REPLACE VIEW workspace.project_gold.vw_delay_causes AS
SELECT
    f.flight_key,
    d.full_date,
    a.airline,
    f.air_system_delay,
    f.security_delay,
    f.airline_delay,
    f.late_aircraft_delay,
    f.weather_delay
FROM workspace.project_gold.FactTable f
LEFT JOIN workspace.project_silver.DimDate d ON f.date_key = d.date_key
LEFT JOIN workspace.project_silver.DimAirlines a ON f.airline_key = a.airline_key;


In [0]:

CREATE OR REPLACE TABLE project_gold.Agg_DailyDelays AS
SELECT
    d.date_key,
    d.full_date,
    AVG(f.departure_delay) AS avg_departure_delay,
    AVG(f.arrival_delay) AS avg_arrival_delay,
    SUM(CASE WHEN f.cancelled = 1 THEN 1 ELSE 0 END) AS cancelled_flights,
    COUNT(*) AS total_flights
FROM project_gold.FactTable f
JOIN project_silver.DimDate d ON f.date_key = d.date_key
GROUP BY d.date_key, d.full_date;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE project_gold.Agg_DailyAirlineTraffic AS
SELECT
    d.date_key,
    d.full_date,
    a.airline,
    COUNT(*) AS total_flights,
    SUM(distance) AS total_distance,
    SUM(CASE WHEN cancelled = 1 THEN 1 END) AS cancelled_flights
FROM project_gold.FactTable f
JOIN project_silver.DimDate d ON f.date_key = d.date_key
JOIN project_silver.DimAirlines a ON f.airline_key = a.airline_key
GROUP BY d.date_key, d.full_date, a.airline;

num_affected_rows,num_inserted_rows


In [0]:
SELECT
    airline,
    AVG(arrival_delay) AS avg_arrival_delay
FROM project_gold.vw_flight_delays
GROUP BY airline
ORDER BY avg_arrival_delay DESC;

airline,avg_arrival_delay
Spirit Air Lines,15.109537361837315
Frontier Airlines Inc.,13.68884881152607
American Eagle Airlines Inc.,7.172238248338438
JetBlue Airways,6.903080015928504
Atlantic Southeast Airlines,6.889618250769496
Skywest Airlines Inc.,6.177718219697895
United Air Lines Inc.,6.1768147942505065
Virgin America,4.9609912831345016
Southwest Airlines Co.,4.811079606259568
American Airlines Inc.,3.910354968930202


Databricks visualization. Run in Databricks to view.

In [0]:
SELECT
    full_date,
    avg_departure_delay
FROM project_gold.Agg_DailyDelays
ORDER BY full_date;

full_date,avg_departure_delay
2015-01-01,9.470546504232106
2015-01-02,12.567921374337905
2015-01-03,24.948126674572848
2015-01-04,31.250592195486846
2015-01-05,20.77332840479711
2015-01-06,22.25289014134511
2015-01-07,14.39946567040987
2015-01-08,15.8618362712231
2015-01-09,15.207929215586184
2015-01-10,8.070180229382851


Databricks visualization. Run in Databricks to view.

In [0]:
SELECT
    ao.airport AS origin_airport,
    AVG(f.departure_delay) AS avg_delay
FROM project_gold.FactTable f
JOIN project_silver.DimAirports ao ON f.origin_airport_key = ao.airport_key
GROUP BY ao.airport
ORDER BY avg_delay DESC
LIMIT 20;

origin_airport,avg_delay
Guam International Airport,22.76104417670683
Wilmington Airport,20.458333333333332
Sawyer International Airport,19.849498327759196
St. Cloud Regional Airport,19.055084745762713
Trenton Mercer Airport,17.35953577410864
Rafael Hernández Airport,17.355838959739934
Plattsburgh International Airport,17.12559241706161
Southwest Oregon Regional Airport (North Bend Municipal),16.82280049566295
Columbia Regional Airport,16.457068062827226
Eagle County Regional Airport,16.305028854080792


Databricks visualization. Run in Databricks to view.

In [0]:
SELECT
    ao.airport AS origin,
    ad.airport AS dest,
    COUNT(*) AS flights
FROM project_gold.FactTable f
JOIN project_silver.DimAirports ao ON f.origin_airport_key = ao.airport_key
JOIN project_silver.DimAirports ad ON f.destination_airport_key = ad.airport_key
GROUP BY origin, dest
ORDER BY flights DESC
LIMIT 20;

origin,dest,flights
Los Angeles International Airport,San Francisco International Airport,40544
San Francisco International Airport,Los Angeles International Airport,39675
Los Angeles International Airport,John F. Kennedy International Airport (New York International Airport),35722
John F. Kennedy International Airport (New York International Airport),Los Angeles International Airport,35717
Los Angeles International Airport,McCarran International Airport,29017
McCarran International Airport,Los Angeles International Airport,28638
Chicago O'Hare International Airport,LaGuardia Airport (Marine Air Terminal),27997
LaGuardia Airport (Marine Air Terminal),Chicago O'Hare International Airport,27777
John F. Kennedy International Airport (New York International Airport),San Francisco International Airport,25052
San Francisco International Airport,John F. Kennedy International Airport (New York International Airport),25039


Databricks visualization. Run in Databricks to view.